[日本語RoBERTa](https://huggingface.co/KoichiYasuoka/roberta-base-japanese-juman-ud-goeswith)による係り受け解析

In [ ]:
!pip install transformers fugashi
import torch,numpy
from transformers import AutoTokenizer,AutoModelForTokenClassification
brt="KoichiYasuoka/roberta-base-japanese-juman-ud-goeswith"
txt="今も海が見えるでしょうか"
tkz=AutoTokenizer.from_pretrained(brt,trust_remote_code=True)
mdl=AutoModelForTokenClassification.from_pretrained(brt)
v,l=tkz(txt,return_offsets_mapping=True),mdl.config.id2label
w=[t for t,(s,e) in zip(v["input_ids"],v["offset_mapping"]) if s<e]
u=[txt[s:e] for s,e in v["offset_mapping"] if s<e]
cls,msk,sep=tkz.cls_token_id,tkz.mask_token_id,tkz.sep_token_id
x=[[cls]+w[:i]+[msk]+w[i+1:]+[sep,j] for i,j in enumerate(w)]
with torch.no_grad():
  m=mdl(input_ids=torch.tensor(x)).logits.numpy()[:,1:-2,:]
r=[1 if i==0 else -1 if l[i].endswith("|root") else 0 for i in range(len(l))]
m+=numpy.where(numpy.add.outer(numpy.identity(m.shape[0]),r)==0,0,numpy.nan)
g=mdl.config.label2id["X|_|goeswith"]
r=numpy.tri(m.shape[0])
for i in range(r.shape[0]):
  for j in range(i+2,r.shape[1]):
    r[i,j]=r[i,j-1] if numpy.nanargmax(m[i,j-1])==g else 1
m[:,:,g]+=numpy.where(r==0,0,numpy.nan)
d,p=numpy.nanmax(m,axis=2),numpy.nanargmax(m,axis=2)
print(" ".join(" "*(9-len(x)*2)+x for x in u))
for i,j in enumerate(u):
  print("\n"+" ".join("{:9.3f}".format(x) for x in d[i])," ",j)
  print(" ".join(l[x].split("|")[0][:9].rjust(9) for x in p[i]))
  print(" ".join(l[x].split("|")[-1][:9].rjust(9) for x in p[i]))

       今        も        海        が    見える  でしょう        か

    0.898    14.462     0.989     0.713     0.920     0.759     0.824   今
     NOUN       ADP      NOUN      NOUN      NOUN      NOUN      NOUN
     root      case       obl       obl      nmod      nmod      nmod

    1.593    -0.015     1.170     0.793     1.264     0.833     0.862   も
     NOUN      VERB      NOUN      NOUN      VERB      NOUN      NOUN
      obl      root       obl       obl     advcl       obl       obl

    4.420     1.380     1.691    14.668     1.245     0.910     1.085   海
     NOUN       ADP      NOUN       ADP      VERB       AUX     PUNCT
      obl      case      root      case       acl       aux     punct

    1.435     0.720     1.436    -0.348     1.503     0.726     0.772   が
     NOUN      NOUN      NOUN      VERB      VERB      NOUN      NOUN
      obl       obl       obl      root     advcl       obl       obl

    9.154     4.225    12.387     1.997    12.444    12.497    12.375   見える
     

In [ ]:
!pip install transformers fugashi deplacy
from transformers import pipeline
brt="KoichiYasuoka/roberta-base-japanese-juman-ud-goeswith"
nlp=pipeline("universal-dependencies",brt,trust_remote_code=True,aggregation_strategy="simple")
doc=nlp("今も海が見えるでしょうか")
import deplacy
deplacy.serve(doc,port=None)

<iframe width="100%" height="400"
src="https://koichiyasuoka.github.io/deplacy/deplacy/editor.html#%23%20text%20%3D%20%E4%BB%8A%E3%82%82%E6%B5%B7%E3%81%8C%E8%A6%8B%E3%81%88%E3%82%8B%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%E3%81%8B%0A1%09%E4%BB%8A%09_%09NOUN%09_%09_%095%09obl%09_%09SpaceAfter%3DNo%0A2%09%E3%82%82%09_%09ADP%09_%09_%091%09case%09_%09SpaceAfter%3DNo%0A3%09%E6%B5%B7%09_%09NOUN%09_%09_%095%09nsubj%09_%09SpaceAfter%3DNo%0A4%09%E3%81%8C%09_%09ADP%09_%09_%093%09case%09_%09SpaceAfter%3DNo%0A5%09%E8%A6%8B%E3%81%88%E3%82%8B%09_%09VERB%09_%09_%090%09root%09_%09SpaceAfter%3DNo%0A6%09%E3%81%A7%E3%81%97%E3%82%87%E3%81%86%09_%09AUX%09_%09_%095%09aux%09_%09SpaceAfter%3DNo%0A7%09%E3%81%8B%09_%09PART%09_%09_%095%09mark%09_%09SpaceAfter%3DNo%0A%0A"
frameborder="0" allowfullscreen>